In [ ]:
from google.colab import files
import os

# make files into a folder

uploaded_files = files.upload()
folder_path = "/content/dataset"

os.makedirs(folder_path, exist_ok=True)

for file_name in uploaded_files.keys():
    os.rename(file_name, os.path.join(folder_path, file_name))

print(f"Files moved to folder: {folder_path}")


In [ ]:
!pip install pytesseract pdf2image pillow

In [ ]:
# Install necessary packages
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!apt-get install -y poppler-utils
!pip install pytesseract pdf2image pillow


In [ ]:
from google.colab import files

# Upload nepali language traineddata file from https://github.com/tesseract-ocr/tessdata/raw/main/nep.traineddata

uploaded = files.upload()
!mv nep.traineddata /usr/share/tesseract-ocr/4.00/tessdata/
!tesseract --list-langs


In [ ]:
import pytesseract
from PIL import Image
import pdf2image
import os

pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

# Function to extract text from an image using Tesseract
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang='nep+eng')  # Using Nepali and English languages
    return text

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    pages = pdf2image.convert_from_path(pdf_path, 300)  
    text = ""
    for page in pages:
        text += pytesseract.image_to_string(page, lang='nep+eng')
    return text

# Function to classify the document
def classify_document(text):
    text = text.lower()

    if "खाता खोल्ने फारम" in text or "account opening form" in text:
        return "Account Opening Form"
    elif "passport" in text:
        return "Passport"
    elif "परिचय पत्र" in text or "permanent account number card" in text:
        return "PAN Card"
    elif "नेपाली नागरिकताको प्रमाणपत्र" in text or "nepali citizenship certificate" in text:
        return "Citizenship Card"
    else:
        return "Unknown Document"

# Main function to process documents in a directory
def process_documents_in_directory(directory_path):
    files = os.listdir(directory_path)

    # Filter out only image and PDF files
    supported_extensions = ['.pdf', '.jpg', '.jpeg', '.png', '.bmp', '.tiff']
    files_to_process = [file for file in files if os.path.splitext(file)[1].lower() in supported_extensions]

    # Process each file
    for file in files_to_process:
        file_path = os.path.join(directory_path, file)
        print(f"Processing {file}...")

        file_extension = os.path.splitext(file)[1].lower()
        if file_extension == '.pdf':
            text = extract_text_from_pdf(file_path)
        elif file_extension in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']:
            text = extract_text_from_image(file_path)
        else:
            print(f"Unsupported file type: {file}")
            continue

        # # Check the extracted text
        # if not text.strip():
        #     print(f"No text extracted from {file}.")
        # else:
        #     print(f"Extracted Text from {file}: {text}")

        # document_type = classify_document(text)
        # print(f"{file} classified as: {document_type}\n")

if __name__ == "__main__":
    directory_path = '/content/dataset'  # Change this to your folder path containing the images or PDFs
    process_documents_in_directory(directory_path)
